## Installing required libraries and Dependencies

In [133]:
!pip install selenium xlsxwriter pandas openpyxl

In [134]:
import time 
from selenium import webdriver # for webscraping
from bs4 import BeautifulSoup  # for webscraping
import xlsxwriter # for converting into excel file 

In [135]:
url = 'https://www.youtube.com/@krishnaik06'

In [ ]:
times = 0
driver = webdriver.Chrome()

driver.get(f"{url}/videos?view=0&sort=p&flow=grid")
while times<5:
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    times +=1
content = driver.page_source.encode('UTF-8').strip()
soup = BeautifulSoup(content,'html.parser')
# soup.prettify()
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import time

# # Initialize WebDriver
# driver = webdriver.Chrome()

# for url in urls:
#     driver.get(f"{url}/videos?view=0&sort=p&flow=grid")
    
#     last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
#     while True:
#         driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
#         time.sleep(2)  # Give time for new content to load
        
#         new_height = driver.execute_script("return document.documentElement.scrollHeight")
#         if new_height == last_height:  # Stop when no new content is loaded
#             break
#         last_height = new_height

#     content = driver.page_source.encode('utf-8').strip()
#     soup = BeautifulSoup(content, 'html.parser')

# soup.prettify()


In [ ]:
t = soup.findAll('a',id="video-title-link") 
titles =[] 
for i in t: 
    titles.append(i.text) 
titles

In [ ]:
# temp=[]
# for string in t:
#     string = string.replace('\n','')
#     temp.append(string)

In [ ]:
# for link in soup.find_all('a'):
#     print(link.get('title'))

In [ ]:
# string = '\n\nGeek for Geek'


In [ ]:
# string=string.replace('\n','')

In [ ]:
# string


In [ ]:

# temp = [x for x in temp if x.strip()]
# temp

In [ ]:
# Views 
v = soup.find_all('span',class_='style-scope ytd-grid-video-renderer')
v

In [ ]:
d = soup.find_all('span', class_ = 'style-scope ytd-thumbnail-overlay-time-status-renderer')
durations = []
for i in d:
    durations.append(i.text.strip())
durations

In [ ]:
v = soup.find_all('a',id = 'video-title-link')
views=[]
for link in v:
    view = link.get('aria-label')
    vlist=view.split(' ')
    views.append(vlist[vlist.index('views')-1])
# view = v.get('aria-label')
# vlist=view.split(' ')
# print(vlist)
# views = vlist[vlist.index('views')-1]
# views
views


In [ ]:
workbook = xlsxwriter.Workbook('file.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0,0,'Title')
worksheet.write(0,1,'Views')
worksheet.write(0,2,'Duration')
row = 1
for title,view,duration in zip(titles,views,durations):
    worksheet.write(row,0,title)
    worksheet.write(row,1,view)
    worksheet.write(row,2,duration)
    row += 1
workbook.close()

In [ ]:
#Data PreProcessing


In [ ]:
#!pip install pandas openpyxl

In [ ]:
import pandas as pd
import openpyxl
df = pd.read_excel('file.xlsx')

In [ ]:
df.head(15)

In [ ]:
df['Views']=df['Views'].str.replace(',','').str.strip()
df['Views']=df['Views'].astype(float)
df['Views']

In [ ]:
df['Views'].map(type)

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
# Duration categories: mini-videos, long-videos,very-long videos 
# converting values into secs
Duration_secs=[]
for i in df['Duration']:
    tim = list(map(int, i.split(':'))) 
    if(len(tim)==1):
        pass
    elif(len(tim)==2):
        tim = tim[0]*60 + tim[1]
    elif(len(tim)==3):
        tim = tim[0]*3600 + tim[1]*60 + tim[2]
    Duration_secs.append(tim)
df['Duration'] = Duration_secs

In [ ]:
df

In [ ]:
# categorize
val_series = df['Duration']
df['Duration'] = df['Duration'].astype(str)

for i in val_series.index:
    val = val_series.iloc[i]
    
    if(val in range(0,900)):
        df.loc[i,'Duration'] = 'Mini-Videos'
    elif(val in range(900,3600)):
        df.loc[i,'Duration'] = 'Long-Videos'
    else:
        df.loc[i,'Duration'] = 'Very-Long-Videos'
    

In [ ]:
# df.loc[0,'Duration']
# df['Duration'].iloc[i] both are same
range(10)

In [ ]:
df

In [ ]:
import re 
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [ ]:
def preprocess_data(text_data):
    preprocessed_text = []
    stop_words = set(stopwords.words("english"))  # Converted to set() for faster lookup (checking membership in a set is much faster than a list).

    for sentence in tqdm(text_data, desc="Processing Text"):  # desc="Processing Text": Adds a label to the progress bar, making it clear what is being processed.
        sentence = re.sub(r"[^\w\s]", "", sentence)  # Remove punctuation,special characters
        filtered_words = [token.lower() for token in sentence.split() if token.lower() not in stop_words]
        preprocessed_text.append(" ".join(filtered_words))
    
    return preprocessed_text

preprocessed_review = preprocess_data(df['Title'].values)
df['Title'] = preprocessed_review


In [ ]:
df['Title']

In [ ]:
#!conda install -c conda-forge wordcloud -y
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
consolidated = ' '.join([word for word in df['Title'].astype(str)])
wordCloud = WordCloud(width=1600, height=800, random_state=21, 
                      max_font_size=110, collocations=False) 

plt.figure(figsize=(15, 10)) 
plt.imshow(wordCloud.generate(consolidated), interpolation='bilinear') 
plt.axis('off')
plt.show() 

In [ ]:
plt.figure(figsize = (5,5))
sns.countplot(data = df, x = "Duration", hue = "Duration", palette = sns.color_palette("hls", 3), legend = False)
plt.title("Number of videos of each category")
plt.xlabel("Duration-wise Category")
plt.ylabel("Count")
plt.show()